### Delete all Project Artifacts

In [ ]:
import json
import boto3
import sagemaker
session = sagemaker.Session()


# Grab admin resources (S3 Bucket name, IAM Roles and Docker Image for Training)
with open('admin_setup.txt', 'r') as filehandle:
    admin_setup = json.load(filehandle)

workflow_name = admin_setup["workflow_name"]
region = session.boto_session.region_name
account_id = boto3.client('sts').get_caller_identity().get('Account')

In [ ]:
!aws s3 rb s3://{workflow_name}-datalake-{account_id} --force
!aws s3 rb s3://{workflow_name}-project-{account_id} --force

In [ ]:
!aws codecommit delete-repository --repository-name {workflow_name}

In [ ]:
!aws lambda delete-function --function-name {workflow_name}-create-sagemaker-training-job
!aws lambda delete-function --function-name {workflow_name}-query-data-processing-status
!aws lambda delete-function --function-name {workflow_name}-create-sagemaker-prcoessing-job
!aws lambda delete-function --function-name {workflow_name}-deploy-sagemaker-model-job
!aws lambda delete-function --function-name {workflow_name}-query-model-accuracy
!aws lambda delete-function --function-name {workflow_name}-codecommit-to-s3
!aws lambda delete-function --function-name {workflow_name}-query-training-status

In [ ]:
state_machine_arn = "arn:aws:states:{}:{}:stateMachine:{}".format(region, account_id, workflow_name)
!aws stepfunctions delete-state-machine --state-machine-arn {state_machine_arn}

In [ ]:
!aws sagemaker delete-endpoint --endpoint-name {workflow_name}